In [ ]:
!pip install -qU langchain langchain-community langchain-classic
!pip install -qU transformers accelerate bitsandbytes sentencepiece gradio faiss-cpu
!pip install -qU torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import gradio as gr
import re
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ======================================================
# 🔴 UPDATED IMPORTS (LangChain 1.0+)
# ======================================================
from langchain_community.llms import HuggingFacePipeline
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationSummaryMemory
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# ======================================================
# 1️⃣ Load LLM
# ======================================================
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

print("🔄 Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Loading Model (4-bit)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

model.eval()
print("✅ LLM loaded")

# We keep 'generation_pipeline' global so the scoring functions can use it directly
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    return_full_text=False  # Prevents repeating the prompt
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

# ======================================================
# 2️⃣ Embeddings
# ======================================================
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# ======================================================
# 3️⃣ Memory Helpers (FIXED)
# ======================================================
BASE_DIR = "memory_store"
os.makedirs(BASE_DIR, exist_ok=True)

def load_faiss(path):
    if os.path.exists(path):
        return FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
    # FIX: Use dummy text ["Initialize"] to avoid dimension error
    return FAISS.from_texts(["Initialize"], embeddings)

def save_faiss(store, path):
    store.save_local(path)

# ======================================================
# 4️⃣ Importance Scoring (0–10) (FIXED)
# ======================================================
def score_importance(text: str) -> int:
    prompt = f"""
Score the importance of the following interaction
for long-term memory from 0 to 10.

Criteria:
- Personal preferences
- Long-term facts
- Goals
- Decisions

Respond with ONLY a number (0-10).

Interaction:
{text}
"""

    # FIX: Use raw pipeline and parse result safely
    result_list = generation_pipeline(prompt)
    result = result_list[0]['generated_text'].strip()

    # Try to find a number in the output (handles "Score: 7" or just "7")
    match = re.search(r'\d+', result)
    if match:
        score = int(match.group())
        return min(10, max(0, score))
    return 0

# ======================================================
# 5️⃣ Memory Type Classification (FIXED)
# ======================================================
def classify_memory(text: str) -> str:
    prompt = f"""
Classify the following interaction.

Respond ONLY with one word:
- fact
- preference
- ignore

Interaction:
{text}
"""

    # FIX: Use raw pipeline
    result_list = generation_pipeline(prompt)
    result = result_list[0]['generated_text'].strip().lower()

    if "fact" in result:
        return "fact"
    if "preference" in result:
        return "preference"
    return "ignore"

# ======================================================
# 6️⃣ User Session Builder (FIXED)
# ======================================================
def get_user_session(user_id: str):
    user_dir = os.path.join(BASE_DIR, user_id)
    os.makedirs(user_dir, exist_ok=True)

    facts_path = os.path.join(user_dir, "facts")
    prefs_path = os.path.join(user_dir, "preferences")

    facts_store = load_faiss(facts_path)
    prefs_store = load_faiss(prefs_path)

    # FIX: Removed return_messages=True to avoid Python object printing
    summary_memory = ConversationSummaryMemory(
        llm=llm
    )

    conversation = ConversationChain(
        llm=llm,
        memory=summary_memory,
        verbose=False
    )

    return conversation, facts_store, prefs_store, facts_path, prefs_path

# ======================================================
# 7️⃣ Chat Logic (Elite Memory System) (FIXED)
# ======================================================
def chat(message, history, user_id):
    conversation, facts_store, prefs_store, facts_path, prefs_path = get_user_session(user_id)

    # Retrieve memories
    facts = facts_store.similarity_search(message, k=2)
    prefs = prefs_store.similarity_search(message, k=2)

    retrieved = "\n".join(
        [d.page_content for d in facts + prefs]
    )

    augmented_input = f"""
Relevant memories:
{retrieved}

User message:
{message}
"""

    response = conversation.predict(input=augmented_input)

    # FIX: Trim hallucinated text (stop loops)
    for stop_string in ["\nUser:", "\nHuman:", "\nAI:"]:
        if stop_string in response:
            response = response.split(stop_string)[0].strip()

    interaction = f"User: {message}\nAssistant: {response}"

    importance = score_importance(interaction)
    memory_type = classify_memory(interaction)

    # Save memory only if important
    if importance >= 6 and memory_type != "ignore":
        if memory_type == "fact":
            facts_store.add_texts([interaction])
            save_faiss(facts_store, facts_path)
        elif memory_type == "preference":
            prefs_store.add_texts([interaction])
            save_faiss(prefs_store, prefs_path)

        print(f"💾 Saved {memory_type} (Score: {importance}) for {user_id}")

    return response

# ======================================================
# 8️⃣ Gradio UI (Multi-User Ready) (FIXED)
# ======================================================
# FIX: Removed theme="soft" to prevent TypeError in new Gradio
demo = gr.ChatInterface(
    fn=chat,
    additional_inputs=[
        gr.Textbox(label="User ID", value="user_1")
    ],
    title="🧠 Smart LLM with Multi-User Memory",
    description="Importance scoring + memory types + FAISS (FREE Colab)",
)

demo.launch(share=True)

🔄 Loading Tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02c9f0d8a9701209d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
